In [1]:
import pandas as pd
import datetime
from collections import Counter

In [2]:
compasdf = pd.read_csv('compas-scores-two-years.csv')

In [3]:
compasdf.columns

Index(['id', 'name', 'first', 'last', 'compas_screening_date', 'sex', 'dob',
       'age', 'age_cat', 'race', 'juv_fel_count', 'decile_score',
       'juv_misd_count', 'juv_other_count', 'priors_count',
       'days_b_screening_arrest', 'c_jail_in', 'c_jail_out', 'c_case_number',
       'c_offense_date', 'c_arrest_date', 'c_days_from_compas',
       'c_charge_degree', 'c_charge_desc', 'is_recid', 'r_case_number',
       'r_charge_degree', 'r_days_from_arrest', 'r_offense_date',
       'r_charge_desc', 'r_jail_in', 'r_jail_out', 'violent_recid',
       'is_violent_recid', 'vr_case_number', 'vr_charge_degree',
       'vr_offense_date', 'vr_charge_desc', 'type_of_assessment',
       'decile_score.1', 'score_text', 'screening_date',
       'v_type_of_assessment', 'v_decile_score', 'v_score_text',
       'v_screening_date', 'in_custody', 'out_custody', 'priors_count.1',
       'start', 'end', 'event', 'two_year_recid'],
      dtype='object')

In [4]:
compasdf = compasdf[compasdf['days_b_screening_arrest'] <= 30]
compasdf = compasdf[compasdf['days_b_screening_arrest'] >= -30]
compasdf = compasdf[compasdf['is_recid'] != -1]
compasdf = compasdf[compasdf['c_charge_degree'] != "O"]
compasdf = compasdf[compasdf['score_text'] != 'N/A']
compasdf['c_jail_in'] = pd.to_datetime(compasdf['c_jail_in'])
compasdf['c_jail_out'] = pd.to_datetime(compasdf['c_jail_out'])
compasdf['length_of_stay'] = (compasdf['c_jail_out'] - compasdf['c_jail_in']).dt.days

In [5]:
compasdf = compasdf[['name','age', 'c_charge_degree', 'race', 'age_cat', 'score_text', 'sex', 'priors_count', 
                    'days_b_screening_arrest', 'decile_score', 'is_recid', 'two_year_recid', 'length_of_stay']]

In [6]:
compasdf.reset_index(drop=True,inplace=True)

In [7]:
compasdf

,name,age,c_charge_degree,race,age_cat,score_text,sex,priors_count,days_b_screening_arrest,decile_score,is_recid,two_year_recid,length_of_stay
0,miguel hernandez,69,F,Other,Greater than 45,Low,Male,0,-1.0,1,0,0,0
1,kevon dixon,34,F,African-American,25 - 45,Low,Male,0,-1.0,3,1,1,10
2,ed philo,24,F,African-American,Less than 25,Low,Male,4,-1.0,4,1,1,1
3,marsha miles,44,M,Other,25 - 45,Low,Male,0,0.0,1,0,0,1
4,edward riddle,41,F,Caucasian,25 - 45,Medium,Male,14,-1.0,6,1,1,6
...,...,...,...,...,...,...,...,...,...,...,...,...,...
6167,steven butler,23,F,African-American,Less than 25,Medium,Male,0,-1.0,7,0,0,1
6168,malcolm simmons,23,F,African-American,Less than 25,Low,Male,0,-1.0,3,0,0,1
6169,winston gregory,57,F,Other,Greater than 45,Low,Male,0,-1.0,1,0,0,1
6170,farrah jean,33,M,African-American,25 - 45,Low,Female,3,-1.0,2,0,0,1


In [8]:
Counter(compasdf.age_cat)

Counter({'Greater than 45': 1293, '25 - 45': 3532, 'Less than 25': 1347})

In [9]:
Counter(compasdf.sex)

Counter({'Male': 4997, 'Female': 1175})

In [10]:
Counter(compasdf.race)

Counter({'Other': 343,
         'African-American': 3175,
         'Caucasian': 2103,
         'Hispanic': 509,
         'Asian': 31,
         'Native American': 11})

In [11]:
Counter(compasdf.score_text)

Counter({'Low': 3421, 'Medium': 1607, 'High': 1144})

# Get factors

In [12]:
crime_factor = pd.get_dummies(compasdf['c_charge_degree'],drop_first=True)
age_factor = pd.get_dummies(compasdf['age_cat'],drop_first=True)
race_factor= pd.get_dummies(compasdf['race'],drop_first=True)
gender_factor = pd.get_dummies(compasdf['sex'],drop_first=True)
score_factor = pd.get_dummies(compasdf['score_text'],drop_first=True)

In [13]:
traindf = compasdf.drop(['c_charge_degree','age_cat','race','score_text','sex'],axis=1)

In [14]:
traindf = pd.concat([traindf,crime_factor,age_factor,race_factor,gender_factor,score_factor],axis=1)

In [15]:
traindf.columns

Index(['name', 'age', 'priors_count', 'days_b_screening_arrest',
       'decile_score', 'is_recid', 'two_year_recid', 'length_of_stay', 'M',
       'Greater than 45', 'Less than 25', 'Asian', 'Caucasian', 'Hispanic',
       'Native American', 'Other', 'Male', 'Low', 'Medium'],
      dtype='object')

In [16]:
# from sklearn.model_selection import train_test_split
# X_train, X_test, y_train, y_test = train_test_split(compasdf.drop(['name','decile_score','is_recid','two_year_recid'],axis=1), 
#                                                     compasdf['two_year_recid'], test_size=0.10, 
#                                                     random_state=101)

In [17]:
from sklearn.linear_model import LogisticRegression
logmodel = LogisticRegression()

data = traindf.drop(['name','decile_score','is_recid','two_year_recid'],axis=1)
target = traindf['two_year_recid']



logmodel.fit(data,target)
predictions = logmodel.predict(data)
probs = logmodel.predict_proba(data)[:,1]

/home/avijit/anaconda3/envs/py37/lib/python3.7/site-packages/sklearn/linear_model/_logistic.py:764: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)


In [28]:
from sklearn.metrics import classification_report,roc_auc_score
print(classification_report(target,predictions))

              precision    recall  f1-score   support

           0       0.68      0.79      0.73      3363
           1       0.69      0.56      0.62      2809

    accuracy                           0.69      6172
   macro avg       0.69      0.68      0.68      6172
weighted avg       0.69      0.69      0.68      6172



In [29]:
log_y_pred = logmodel.predict_proba(data)
log_roc = roc_auc_score(target,log_y_pred[:,-1])
print('ROC AUC score : ',log_roc)
print(logmodel.get_params())

ROC AUC score :  0.7425658171289409
{'C': 1.0, 'class_weight': None, 'dual': False, 'fit_intercept': True, 'intercept_scaling': 1, 'l1_ratio': None, 'max_iter': 100, 'multi_class': 'auto', 'n_jobs': None, 'penalty': 'l2', 'random_state': None, 'solver': 'lbfgs', 'tol': 0.0001, 'verbose': 0, 'warm_start': False}


In [19]:
compasdf = pd.concat([compasdf,pd.Series(probs).rename('two_year_recid_pred')],axis=1)
compasdf = compasdf.drop(['decile_score','is_recid'],axis=1)

In [20]:
def racebin(race):
    if race == 'Caucasian':
        return 'White'
    else:
        return 'NonWhite'

In [21]:
compasdf['race_binary'] = compasdf['race'].apply(racebin)

In [22]:
race_factor= pd.get_dummies(compasdf['race_binary'],drop_first=True)
gender_factor = pd.get_dummies(compasdf['sex'],drop_first=True)

In [23]:
compasdf = compasdf.drop(['race','sex','race_binary'],axis=1)
compasdf = pd.concat([compasdf,race_factor,gender_factor],axis=1)

In [24]:
#compasdf.to_csv('compas_baseline_score.csv',index=False)

In [25]:
compasdf

,name,age,c_charge_degree,age_cat,score_text,priors_count,days_b_screening_arrest,two_year_recid,length_of_stay,two_year_recid_pred,White,Male
0,miguel hernandez,69,F,Greater than 45,Low,0,-1.0,0,0,0.135285,0,1
1,kevon dixon,34,F,25 - 45,Low,0,-1.0,1,10,0.308855,0,1
2,ed philo,24,F,Less than 25,Low,4,-1.0,1,1,0.560977,0,1
3,marsha miles,44,M,25 - 45,Low,0,0.0,0,1,0.214670,0,1
4,edward riddle,41,F,25 - 45,Medium,14,-1.0,1,6,0.787963,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...
6167,steven butler,23,F,Less than 25,Medium,0,-1.0,0,1,0.549013,0,1
6168,malcolm simmons,23,F,Less than 25,Low,0,-1.0,0,1,0.437680,0,1
6169,winston gregory,57,F,Greater than 45,Low,0,-1.0,0,1,0.180786,0,1
6170,farrah jean,33,M,25 - 45,Low,3,-1.0,0,1,0.285965,0,0
